# 한글-Claude-v2 Model: Conversational Interface - Chatbot with Claude LLM

> 이 노트북은 SageMaker Studio의 **`Data Science 3.0`** 커널과 잘 작동합니다.

> **SageMaker Notebook Instance** 를 이용해 실습을 진행하신다면, **JupyterLab이 아닌 Jupyter**에서 실행하시기 바랍니다.

이 노트북에서는 Amazon Bedrock의 기본 모델 (FM) 을 사용하여 챗봇을 구축할 것입니다.사용 사례에서는 Claude를 챗봇 구축을 위한 FM으로 사용합니다.

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## 개요

챗봇 및 가상 어시스턴트와 같은 대화형 인터페이스를 사용하여 고객의 사용자 경험을 향상시킬 수 있습니다. 챗봇은 자연어 처리 (NLP) 및 기계 학습 알고리즘을 사용하여 사용자 쿼리를 이해하고 이에 응답합니다.챗봇은 고객 서비스, 판매, 전자 상거래와 같은 다양한 애플리케이션에서 사용되어 사용자에게 빠르고 효율적인 응답을 제공할 수 있습니다.웹사이트, 소셜 미디어 플랫폼 및 메시징 앱과 같은 다양한 채널을 통해 액세스할 수 있습니다.


## Amazon Bedrock을 사용하는 챗봇

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## 사용 사례
- 1.**챗봇 (기본)** - FM 모델을 사용하는 제로샷 챗봇
- 2.**프롬프트를 사용하는 챗봇** - 템플릿 (Langchain) - 프롬프트 템플릿에 일부 컨텍스트가 제공된 챗봇
- 3.**페르소나가 있는 챗봇** - 정의된 역할을 가진 챗봇. 즉, 커리어 코치와 인간 상호작용
- 4.**컨텍스트 인식 챗봇** - 임베딩을 생성하여 외부 파일을 통해 컨텍스트를 전달합니다.

## Amazon Bedrock으로 챗봇을 구축하기 위한 랭체인 프레임워크
챗봇과 같은 대화형 인터페이스에서는 단기적 수준뿐만 아니라 장기적 수준에서도 이전 상호 작용을 기억하는 것이 매우 중요합니다.

LangChain은 메모리 구성 요소를 두 가지 형태로 제공합니다.먼저 LangChain은 이전 채팅 메시지를 관리하고 조작하기 위한 도우미 유틸리티를 제공합니다.모듈식으로 설계되어 사용 방식에 관계없이 유용하게 사용할 수 있습니다.둘째, LangChain은 이러한 유틸리티를 체인에 통합하는 쉬운 방법을 제공합니다.
이를 통해 다양한 유형의 추상화를 쉽게 정의하고 상호 작용할 수 있으므로 강력한 챗봇을 쉽게 구축할 수 있습니다.

## 컨텍스트를 활용한 챗봇 구축하기 - 핵심 요소

컨텍스트 인식 챗봇을 구축하는 첫 번째 프로세스는 컨텍스트에 대한**임베딩을 생성**하는 것입니다.일반적으로 임베딩 모델을 통해 실행되고 일종의 벡터 저장소에 저장될 임베딩을 생성하는 통합 프로세스가 있습니다.이 예제에서는 이를 위해 GPT-J 임베딩 모델을 사용하고 있습니다.

![Embeddings](./images/embeddings_lang.png)

두 번째 프로세스는 사용자 요청 오케스트레이션, 상호 작용, 호출 및 결과 반환입니다.

![Chatbot](./images/chatbot_lang.png)

## 아키텍처 [컨텍스트 인식 챗봇]
![4](./images/context-aware-chatbot.png)


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.


이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [FAISS](https://github.com/facebookresearch/faiss), 벡터 임베딩 저장용
- [iPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), 노트북의 대화형 UI 위젯용
- [PyPDF](https://pypi.org/project/pypdf/), PDF 파일 처리용

In [2]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.249 "pypdf>=3.8,<4"

Note: you may need to restart the kernel to use updated packages.


# 1. Bedrock Client 생성

In [3]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


# 2. 챗봇(기본 - 컨텍스트 없음)

우리는 LangChain의 [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain)을 사용하여 시작합니다. 대화. 또한 메시지 저장을 위해 [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html)를 사용합니다. 메시지 목록으로 기록을 얻을 수도 있습니다(채팅 모델에서 매우 유용합니다).

챗봇은 이전 상호작용을 기억해야 합니다. 대화 기억을 통해 우리는 그렇게 할 수 있습니다. 대화형 메모리를 구현하는 방법에는 여러 가지가 있습니다. LangChain의 맥락에서 이들은 모두 ConversationChain 위에 구축됩니다.

**참고:** 모델 출력은 비결정적입니다.

In [4]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory

cl_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="안녕하세요?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요?
AI:

> Finished chain.
 네, 안녕하세요! 반갑습니다.


### 결과 분석

여기서 무슨 일이 일어나는가? 우리는 "안녕하세요!"라고 말했습니다. 모델은 몇 가지 대화를 나눴습니다. 이는 Langchain ConversationChain에서 사용하는 기본 프롬프트가 Claude에 맞게 잘 설계되지 않았기 때문입니다. 
- [효과적인 클로드 프롬프트](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)는 `\n\nHuman\n\nAassistant:`로 끝나야 합니다. 이 문제를 해결해 보겠습니다.

Claude의 프롬프트 작성 방법에 대해 자세히 알아보려면 [Anthropic 문서](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)를 확인하세요.

# 3. 프롬프트 템플릿(Langchain)을 이용한 챗봇

LangChain은 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 기능을 제공합니다. [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) 클래스를 사용하여 f-string 템플릿에서 프롬프트를 구성하겠습니다.

In [5]:
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude

claude_prompt = PromptTemplate.from_template("""다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
{history}


Human: {input}


Assistant:
""")


print("claude_prompt: \n", claude_prompt)

claude_prompt: 
 input_variables=['history', 'input'] output_parser=None partial_variables={} template='다음은 사람과 인공지능의 친근한 대화입니다. \n인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. \n인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.\n\nCurrent conversation:\n{history}\n\n\nHuman: {input}\n\n\nAssistant:\n' template_format='f-string' validate_template=True


In [6]:
conversation.prompt = claude_prompt

print_ww(conversation.predict(input="안녕하세요?"))

 안녕하세요! 만나서 반갑습니다.


#### (1) 새로운 질문

모델이 초기 메시지로 응답했습니다. 몇 가지 질문을 해보겠습니다.

In [7]:
print_ww(conversation.predict(input="새로운 정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요."))

 새로운 정원을 시작할 때 다음과 같은 팁이 도움이 될 수 있습니다:

- 먼저 정원의 위치, 크기, 활용 목적 등을 고려하세요. 해당 위치의 기후, 토양, 일조량 등을 파악하고 정원에 맞는 식물을 선택하세요.

- 토양을 건강하게 유지하기 위해 주기적으로 퇴비를 추가하고, 질소 고정 식물을 심어 토양의 질을 높이세요.

- 물 공급을 위한 시스템(계획)을 마련하세요. 배수와 침투를 위해 정원의 경사를 고려하세요.

- 정원의 레이아웃을 계획하세요. 꽃, 채소, 허브 등을 구역별로 구분하는 것이 좋습니다.

- 초기에는 잡초를 자주 제거하고, 병해충을 예방하기 위해 유기농 방제를 활용하세요.

- 수시로 관찰하고 기록으로 관리하며, 식물의 생육환경을 최적화하기 위해 노력하세요.

- 적당한 규모로 시작하고 천천히 확장하는 것이 좋습니다. 인내심을 가지고 꾸준히 관리하세요.

정성을 들여 정원을 가꾸다 보면 조화로운 정원을 창조할 수 있습니다. 즐거운 정원 가꾸기 되세요!


#### (2) 질문을 토대로 작성

모델이 이전 대화를 이해할 수 있는지 확인하기 위해 정원이라는 단어를 언급하지 않고 질문해 보겠습니다.

In [8]:
print_ww(conversation.predict(input="좋아요. 토마토에도 어울릴까요?"))

 네, 토마토도 새로운 정원에 잘 어울리는 작물 중 하나입니다.

토마토 정원을 가꾸기 위한 팁을 몇 가지 더 드리겠습니다:

- 토마토는 일조량이 많고 환기가 잘 되는 곳을 선호합니다.

- 토양은 수분 배출이 잘 되면서도 영양분을 공급할 수 있는 양토가 적당합니다.

- 지주를 설치하여 덩굴성 토마토를 묶어주거나, 이삭 토마토는 가지를 지탱해주세요.

- 정기적인 시비를 통해 토양 영양분을 공급하세요.

- 수확기에는 균일한 수분 공급이 중요합니다. 관수 시스템을 마련하세요.

- 병충해 예방을 위해 주기적으로 유기농 방제를 실시하세요.

- 토마토에 맞는 품종을 선택하세요. 지역의 기후와 재배 환경에 따라 다릅니다.

토마토의 성장환경을 잘 조성하면 풍성한 수확을 기대할 수 있습니다. 좋은 토마토 정원 만드시기 바랍니다!


#### (3) 대화를 마치며

In [9]:
print_ww(conversation.predict(input="그게 다야, 고마워!"))

 네, 처음 정원 가꾸기를 시작하시는 분을 위해 새로운 정원 시작 팁과 토마토 정원 가꾸기 팁을 간략히 설명드렸습니다. 제가 알려줄 수 있는 정보로 충분히 도움이 되었기를
바랍니다. 정원 가꾸기 잘 하시고 풍성한 수확이 있기를 기원합니다!


# 4. ipywidgets를 사용한 대화형 세션

다음 유틸리티 클래스를 사용하면 Claude와 보다 자연스러운 방식으로 상호 작용할 수 있습니다. 입력창에 질문을 적고 클로드의 답변을 받습니다. 그러면 대화를 계속할 수 있습니다.

In [10]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

#### (1) 채팅을 시작해 보겠습니다. 다음 질문을 테스트할 수도 있습니다.

1. 농담 하나 해줘
2. 또 다른 농담을 들려주세요
3. 첫 번째 농담은 무엇이었나요?
4. 첫 번째 농담과 같은 주제로 또 다른 농담을 할 수 있나요?

위의 4가지를 순서대로 아래에 입력하시고, "Send" 버튼을 눌러 보세요.

In [11]:
chat = ChatUX(conversation)
chat.start_chat()

Starting chat bot


Output()

# 5.페르소나를 활용한 챗봇

AI 비서가 커리어 코치 역할을 하게 됩니다. 
- 역할극 대화에서는 채팅을 시작하기 전에 사용자 메시지를 설정해야 합니다. ConversationBufferMemory는 대화 상자를 미리 채우는 데 사용됩니다.

### (1) ConversationChain 생성 필요한 메모리 초기화, Bedrock Claude 설정

In [12]:
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다")
memory.chat_memory.add_ai_message("나는 직업 코치이며 직업에 대한 조언을 제공합니다")
cl_llm = Bedrock(model_id="anthropic.claude-v2",
                 client=boto3_bedrock,
                 model_kwargs={"max_tokens_to_sample": 1000},                 
                )
conversation = ConversationChain(
     llm=cl_llm, verbose=True, memory=memory
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
{history}


Human: {input}


Assistant:
""")


print("claude_prompt: \n", claude_prompt)
conversation.prompt = claude_prompt

claude_prompt: 
 input_variables=['history', 'input'] output_parser=None partial_variables={} template='다음은 사람과 인공지능의 친근한 대화입니다. \n인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. \n인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.\n\nCurrent conversation:\n{history}\n\n\nHuman: {input}\n\n\nAssistant:\n' template_format='f-string' validate_template=True


### (2) 인공지능 관련 직업 질문

In [13]:
# print_ww(conversation.predict(input="“AI의 직업 종류는 무엇입니까?"))
print_ww(conversation.predict(input="“인공지능에 관련된 직업은 어떤 것이 있습니까?"))



> Entering new ConversationChain chain...
Prompt after formatting:
다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
Human: 당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다
AI: 나는 직업 코치이며 직업에 대한 조언을 제공합니다


Human: “인공지능에 관련된 직업은 어떤 것이 있습니까?


Assistant:


> Finished chain.
 인공지능과 관련된 주요 직업으로는 다음과 같습니다:

- 인공지능 연구원 - 인공지능 알고리즘과 기술을 연구하고 개발합니다. 수학, 통계, 컴퓨터 공학 등의 배경 지식이 필요합니다.

- 머신러닝 엔지니어 - 머신러닝 모델을 설계, 구축, 테스트하고 실제 제품/서비스에 적용합니다. 파이썬, 텐서플로 등의 기술 숙련도가 필요합니다.

- 데이터 사이언티스트 - 기업의 데이터를 분석하고 인사이트를 도출하며, 머신러닝 모델을 구축합니다. 통계, 데이터 마이닝, 프로그래밍 능력이 필요합니다.

- NLP(자연어처리) 엔지니어 - 자연어를 처리하고 이해하는 인공지능 시스템을 개발합니다. 자연어 처리, 자연어 생성, 음성 인식 등의 업무를 수행합니다.

- 컴퓨터 비전 엔지니어 - 이미지와 동영상을 인식하고 이해하는 인공지능 시스템을 개발합니다. 딥러닝, 컴퓨터 비전 알고리즘 설계 능력이 필요합니다.

- 로봇공학자 - 센서를 장착한 인공지능 로봇의 하드웨어와 소프트웨어를 설계합니다. 기계공학, 로봇틱스, 프로그래밍 등의 지식이 필요합니다.

이외에도 인공지능 컨설턴트, 인공지능 기획자 등 다양한 인공지능 관련 직업이 있습니다. 수학과 공학 실력과 함께 창의력이 필요한 분야입니다.


### (3) 인공지능 관련 직업데 대한 세부 질문

In [14]:
print_ww(conversation.predict(input="이 직업들은 실제로 무엇을 하는가요? 재미있나요?"))



> Entering new ConversationChain chain...
Prompt after formatting:
다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
Human: 당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다
AI: 나는 직업 코치이며 직업에 대한 조언을 제공합니다
Human: “인공지능에 관련된 직업은 어떤 것이 있습니까?
AI:  인공지능과 관련된 주요 직업으로는 다음과 같습니다:

- 인공지능 연구원 - 인공지능 알고리즘과 기술을 연구하고 개발합니다. 수학, 통계, 컴퓨터 공학 등의 배경 지식이 필요합니다. 

- 머신러닝 엔지니어 - 머신러닝 모델을 설계, 구축, 테스트하고 실제 제품/서비스에 적용합니다. 파이썬, 텐서플로 등의 기술 숙련도가 필요합니다.

- 데이터 사이언티스트 - 기업의 데이터를 분석하고 인사이트를 도출하며, 머신러닝 모델을 구축합니다. 통계, 데이터 마이닝, 프로그래밍 능력이 필요합니다. 

- NLP(자연어처리) 엔지니어 - 자연어를 처리하고 이해하는 인공지능 시스템을 개발합니다. 자연어 처리, 자연어 생성, 음성 인식 등의 업무를 수행합니다.

- 컴퓨터 비전 엔지니어 - 이미지와 동영상을 인식하고 이해하는 인공지능 시스템을 개발합니다. 딥러닝, 컴퓨터 비전 알고리즘 설계 능력이 필요합니다.

- 로봇공학자 - 센서를 장착한 인공지능 로봇의 하드웨어와 소프트웨어를 설계합니다. 기계공학, 로봇틱스, 프로그래밍 등의 지식이 필요합니다.

이외에도 인공지능 컨설턴트, 인공지능 기획자 등 다양한 인공지능 관련 직업이 있습니다. 수학과 공학 실력과 함께 창의력이 필요한 분야입니다.


Human: 이 직업들은 실제로 무엇을 하는가요? 재미있나요?


Assistant:


> Finish

# 6. 맥락을 가진 챗봇

## 상황에 맞는 챗봇
이 사용 사례에서는 Chatbot에게 이전에 본 적이 없는 외부 코퍼스의 질문에 답변하도록 요청합니다. 이를 위해 RAG(Retrieval Augmented Generation)라는 패턴을 적용합니다. 아이디어는 말뭉치를 덩어리로 인덱싱한 다음 덩어리와 질문 사이의 의미론적 유사성을 사용하여 말뭉치의 어느 섹션이 답변을 제공하는 데 관련될 수 있는지 찾는 것입니다. 마지막으로 가장 관련성이 높은 청크가 집계되어 기록을 제공하는 것과 유사하게 ConversationChain에 컨텍스트로 전달됩니다.

**Titan Embeddings Model**을 사용하여 벡터를 생성하겠습니다. 그런 다음 이 벡터는 메모리 내 벡터 데이터 저장소를 제공하는 오픈 소스 라이브러리인 FAISS에 저장됩니다. 챗봇이 질문을 받으면 FAISS에 질문을 쿼리하고 의미상 가장 가까운 텍스트를 검색합니다. 이것이 우리의 대답이 될 것입니다.

## 6.1 2022년 아마존 주주 서한 문서로 구현 (PDF 문서)

### Amazon Titan Embedding 모델 사용 
- model_id='amazon.titan-e1t-medium'
- 이 모델은 최대 512 Token 입력이 가능합니다. 추후에 최대 토큰 사이즈가 큰 모델이 나오면 바꾸어서 테스트하시기 바랍니니다. 
- 일반적으로 한글 임베딩시에 512 토큰은 작습니다.

In [15]:
from langchain.embeddings import BedrockEmbeddings

bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)
bedrock_embeddings

BedrockEmbeddings(client=<botocore.client.Bedrock object at 0x7f10d75ae8c0>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-e1t-medium', model_kwargs=None, endpoint_url=None)

### PyPDFDirectoryLoader 를 통한 PDF 파일 로딩
- chunk_size 는 임베딩 모델의 최대 입력 토큰이 512를 고려 해서 정했습니다. 아래 수치보다 증가시킬 경우에 Embedding Model 에 벡터 변환을 요구할 시에, 토큰이 512 보다 커서 에러가 발생합니다

In [16]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = PyPDFDirectoryLoader("rag_data_kr_pdf/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 230,
    chunk_overlap  = 50,
#    separators = ['\n']
#    separators = ['\n','\n\n']    
)
docs = text_splitter.split_documents(documents)

In [17]:
%%time
vectorstore_faiss_aws = FAISS.from_documents(
    documents=docs,
    embedding = bedrock_embeddings
)

print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)

vectorstore_faiss_aws: number of elements in the index=92::
CPU times: user 359 ms, sys: 30.3 ms, total: 389 ms
Wall time: 16.7 s


In [18]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 10 documents loaded is 1606 characters.
After the split we have 92 documents more than the original 10.
Average length among 92 documents (after split) is 216 characters.


In [19]:
print("docs[0].page_content: \n", docs[0].page_content)

docs[0].page_content: 
 CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다. 2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해). 우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을


In [20]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [-0.2734375   0.07958984  0.15136719 ...  0.44335938 -0.10546875
  0.14355469]
Size of the embedding:  (4096,)


### Semantic search

LangChain에서 제공하는 Wrapper 클래스를 사용하여 벡터 데이터베이스 저장소를 쿼리하고 관련 문서를 반환할 수 있습니다. 뒤에서는 RetrievalQA 체인만 실행됩니다.

In [21]:
query = "아마존은 GeneraTve AI 의 전략이 무엇인가요?"
print_ww(wrapper_store_faiss.query(query, llm=cl_llm))

 죄송합니다, 제한된 정보로는 아마존의 GeneraTve AI 전략에 대해 알기 어렵습니다. 아마존의 구체적인 상세 정보 없이는 이 질문에 정확하게 답변할 수 없습니다. 아마존의
AI 연구 및 개발에 대한 공개 정보가 더 있다면 그 정보를 바탕으로 답변을 제공할 수 있겠지만, 현재로서는 잘 모르겠습니다. 죄송합니다.


의미론적 검색이 어떻게 작동하는지 살펴보겠습니다.
1. 먼저 쿼리에 대한 임베딩 벡터를 계산하고
2. 그런 다음 이 벡터를 사용하여 벡터 스토어에서 유사성 검색을 수행합니다.

In [22]:
v = bedrock_embeddings.embed_query(query)
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=3)
for r in results:
    print_ww(r.page_content)
    print('----')

[0.08984375, -0.25195312, 0.23242188, 0.2890625, -0.22460938, 0.24707031, 0.18066406, -0.0017623901, -0.375, 0.22851562]
학습 서비스(AWS가 가장 광범위한 기계를 보유한 곳) 모든 클라우드 공급자의 학습 기능 및 고객 기반). 보다 최근에는 GeneraTve AI라고 하는 새로운 형태의 기계 학습이
등장하여 기계 학습 채택을 크게 가속화할 것이라고 약속합니다. 제너레이티브 AI는 매우 큰 언어 모델(최대 천억 개의 매개변수에 대해 훈련되고 계속 증가하고 있음)을 기반으로 하며
광범위한 데이터 세트에 걸쳐 매우 일반적이고
----
LLM과 GeneraTve AI가 그렇게 변혁적일 것이라고 생각하기 때문에 전체 편지를 쓸 수 있지만 향후 편지로 남겨두겠습니다. LLM과 GeneraTve AI가 고객, 주주 및
Amazon에게 큰 문제가 될 것이라고 가정해 보겠습니다.  그래서 마지막으로 저는 우리가 이 도전적인 거시경제 시대에 진입했을 때보다 더 강력한 위치에서 등장할 것이라고
낙관합니다. 여기에는 몇 가지 이유가 있으며 위에서 많은
----
실행할 수 있습니다. 우리는 회사가 다양한 LLM 중에서 선택하고 고객이 사용하는 데 익숙한 모든 AWS 보안, 개인 정보 보호 및 기타 기능을 갖춘 애플리케이션을 구축할 수
있도록 합니다. 또한 실시간으로 코드 제안을 생성하여 개발자 생산성을 혁신하는 AWS의 CodeWhisperer와 같은 애플리케이션을 제공하고 있습니다. LLM과 GeneraTve
AI가 그렇게 변혁적일 것이라고 생각하기 때문에 전체
----


### 메모리
모든 챗봇에는 사용 사례에 따라 맞춤화된 다양한 옵션을 갖춘 QA 체인이 필요합니다. 그러나 챗봇에서는 모델이 답변을 제공하기 위해 이를 고려할 수 있도록 항상 대화 기록을 보관해야 합니다. 이 예에서는 대화 기록을 유지하기 위해 ConversationBufferMemory와 함께 LangChain의 [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db)을 사용합니다.

출처: https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db

뒤에서 무슨 일이 일어나고 있는지 모두 보려면 'verbose'를 'True'로 설정하세요.

In [23]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a
standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


### ConversationRetrievalChain에 사용되는 매개변수
* **retriever**: 우리는 `VectorStore`가 지원하는 `VectorStoreRetriever`를 사용했습니다. 텍스트를 검색하려면 `"similarity"` 또는 `"mmr"`라는 두 가지 검색 유형을 선택할 수 있습니다. `search_type="similarity"`는 질문 벡터와 가장 유사한 텍스트 청크 벡터를 선택하는 검색 객체에서 유사성 검색을 사용합니다.

* **메모리**: 이력을 저장하는 메모리 체인

* **dense_question_prompt**: 사용자의 질문이 주어지면 이전 대화와 해당 질문을 사용하여 독립형 질문을 구성합니다.

* **chain_type**: 채팅 기록이 길고 상황에 맞지 않는 경우 이 매개변수를 사용하고 옵션은 `stuff`, `refine`, `map_reduce`, `map-rerank`입니다.

질문이 컨텍스트 범위를 벗어나면 모델은 답을 모른다고 대답합니다.

**참고**: 체인이 어떻게 작동하는지 궁금하다면 `verbose=True` 줄의 주석 처리를 해제하세요.

In [24]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

채팅을 해보죠. 아래와 같은 질문을 해보세요. 
1. 아마존은 GeneraTve AI 의 전략이 무엇인가요?

In [25]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

이는 이 노트북의 시작 부분에 설명된 것과 동일한 이유로 발생합니다. 기본 랭체인 프롬프트는 Claude에게 최적이 아닙니다.
다음 셀에서는 두 개의 새로운 프롬프트를 설정하겠습니다. 하나는 질문의 표현을 변경하기 위한 것이고, 다른 하나는 해당 질문의 답변을 얻기 위한 것입니다.

In [26]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

새로운 질문으로만 대답하세요.


Human: 이전 대화를 고려하여 질문을 어떻게 하시겠습니까?: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provide longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v1", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}


Human: <q></q> XML 태그 내의 질문에 답하려면 최대 3개의 문장을 사용하세요.

<q>{question}</q>

답변에 XML 태그를 사용하지 마세요. 답변이 context에 없으면 "죄송합니다. 문맥에서 답을 찾을 수 없어서 모르겠습니다."라고 말합니다.

Assistant:""")

채팅을 해보죠. 아래와 같은 질문을 해보세요. 
1. 아마존은 GeneraTve AI 의 전략이 무엇인가요?

In [27]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

## 6.2. 신한은행 FAQ 데이터 세트로 구현

### 데이터 로딩 및 확인

In [28]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "./rag_data_kr_csv/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

,no,question,context,type,source
0,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
1,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
2,87,금융인증서 해외에서 발급할 수 있나요?,"해외에서도 인증서 발급 업무는 가능합니다. 다만, 금융인증서 저장을 위한 금융결제원...",금융인증서,신한은행
3,86,클라우드에 보관 중인 인증서는 얼마나 유지 되나요?,"정상적으로 이용하실 경우 금융인증서의 유효기간은 3년이며, 1년 동안 클라우드 이용...",금융인증서,신한은행
4,85,금융인증서 발급대상은 누구인가요?,금융인증서는 은행 등의 금융회사에서 실명 확인된 고객을 대상으로 안전하게 발급되며 ...,금융인증서,신한은행
...,...,...,...,...,...
84,5,인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요,"인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고...",인터넷뱅킹,신한은행
85,4,타행으로 자동이체를 신청하고 싶은데요,인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는...,인터넷뱅킹,신한은행
86,3,공과금 자동이체 신청이 가능한가요?,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동...",인터넷뱅킹,신한은행
87,2,인터넷뱅킹 거래에 대한 책임 범위를 알고싶습니다.,기본적으로 은행이 인터넷뱅킹 사고에 대한 책임을 집니다. 해커의 침입에 의해 자금이...,인터넷뱅킹,신한은행


### 데이터 전처리
- 여기서 데이터 컬럼인 context 만을 사용 합니다.
- char_size <= 210 인 데이터만 사용합니다. (Embedding Model 최대 입력 토큰이 512 여서 제한을 둡니다.)

In [29]:
def preprocess_data(df, char_size):
    ldf = df.copy()
    ldf['Len'] = ldf['context'].apply(lambda x : len(x))
    ldf = ldf[ldf['Len'] < char_size]

    df_index = ldf.drop(['no','question','type','source','Len'], axis=1)
    # df_index.to_csv("rag_data_kr/fsi_smart_faq_ko_answer.csv", index=None, header=None)
    df_index.to_csv("rag_data_kr_csv/fsi_smart_faq_ko_answer.csv", index=None)

    return df_index

pre_df = preprocess_data(df, char_size=210)
pre_df

,context
2,"해외에서도 인증서 발급 업무는 가능합니다. 다만, 금융인증서 저장을 위한 금융결제원..."
3,"정상적으로 이용하실 경우 금융인증서의 유효기간은 3년이며, 1년 동안 클라우드 이용..."
4,금융인증서는 은행 등의 금융회사에서 실명 확인된 고객을 대상으로 안전하게 발급되며 ...
5,공동인증서와 금융인증서는 별개의 인증서로 두 가지 인증서를 모두 사용할 수 있습니다.
6,"뱅크아이디란, 블록체인기반의 은행권 공동인증서비스로 블록체인 원장을 분산하여 저장/..."
...,...
80,"인터넷뱅킹에 로그인 하신 후 ""사용자관리""를 클릭하신 후 ""고객정보"" > ""고객정보..."
82,인터넷상으로 예금/신탁을 신규가입하시려면 우선 고객님께서는인터넷뱅킹에 가입하셔야 하...
83,인터넷뱅킹(bank.shinhan.com) 송금 수수료는 우측 퀵메뉴 이용안내 수수...
86,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동..."


### 문서 로딩 및 청킹

In [30]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import FAISS

loader = CSVLoader("rag_data_kr_csv/fsi_smart_faq_ko_answer.csv", source_column="context") # --- > 219 docs with 400 chars, each row consists in a question column and an answer column
documents_fsi = loader.load() #
print(f"Number of documents={len(documents_fsi)}")


text_splitter = CharacterTextSplitter(chunk_size=210, 
                                      chunk_overlap=20,
                                      #separator=","
                                     )

docs = text_splitter.split_documents(documents_fsi)

print(f"Number of documents after split and chunking={len(docs)}")

Number of documents=43
Number of documents after split and chunking=43


### 문서 사이즈 등 통계

In [31]:

def show_chunk_stat(documents):
    doc_len_list = [len(doc.page_content) for doc in documents]
    print(pd.DataFrame(doc_len_list).describe())
    avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
    avg_char_count_pre = avg_doc_length(documents)
    print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
    
    max_idx = doc_len_list.index(max(doc_len_list))
    print("\nShow document at maximum size")
    print(documents[max_idx].page_content)    
    
show_chunk_stat(docs)

                0
count   43.000000
mean   143.116279
std     52.944360
min     33.000000
25%    103.500000
50%    142.000000
75%    196.000000
max    214.000000
Average length among 43 documents loaded is 143 characters.

Show document at maximum size
context: 인터넷뱅킹 가입과 홈페이지 회원은 개별로 운영되고 있습니다. 
인터넷뱅킹만 가입을 하셨다면 별도로 홈페이지에 회원가입 후 홈페이지 로그인이 가능합니다. 
※ 영업점에서 인터넷뱅킹 가입 시 전계좌조회서비스 가입하신 고객은 홈페이지 상에서 이용자비밀번호 등록 후 이용가능합니다. 
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.


### FAISS Vector Store 생성

In [32]:
%%time
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
vectorstore_faiss_fsi = FAISS.from_documents(
    documents=docs,
    embedding = bedrock_embeddings
)

print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_fsi.index.ntotal}::")
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_fsi)

vectorstore_faiss_aws: number of elements in the index=43::
CPU times: user 153 ms, sys: 11.1 ms, total: 164 ms
Wall time: 6.15 s


In [33]:
print("docs[0].page_content: \n", docs[0].page_content)

docs[0].page_content: 
 context: 해외에서도 인증서 발급 업무는 가능합니다. 다만, 금융인증서 저장을 위한 금융결제원 클라우드 계정 생성 및 연결이 필요한 업무로 해외연락처를 통한 ARS인증이 진행됩니다.


#### Semantic search

LangChain에서 제공하는 Wrapper 클래스를 사용하여 벡터 데이터베이스 저장소를 쿼리하고 관련 문서를 반환할 수 있습니다. 뒤에서는 RetrievalQA 체인만 실행됩니다.

In [34]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: 다음 문맥을 사용하여 마지막 질문에 대한 간결한 답변을 제공하세요. 답을 모르면 모른다고 말하고 답을 만들어내려고 하지 마세요.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=cl_llm,
    chain_type="stuff",
    retriever=vectorstore_faiss_fsi.as_retriever(
        search_type="similarity", search_kwargs={"k": 5}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)


#query = "증권사에서 발급받은 인증서는 은행에서 사용 불가한가요?"
# query = "인터넷뱅킹 자동이체 신청 및 해지 방법을 알려주세요"
query = "신한은행 고객센터 전화 번호 알려줘?"
result = qa({"query": query})
print_ww(result['result'])

 신한은행 고객센터 전화번호는 1599-8000입니다.


In [35]:
def show_context_used(context_list):
    for idx, context in enumerate(context_list):
        print("-----------------------------------------------")                
        print(f"{idx+1}. context to be fed into FM(e.g. Claude-v2)")
        print("-----------------------------------------------")        
        print_ww(context.page_content)        
        
show_context_used(result['source_documents'])        

-----------------------------------------------
1. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
context: 통합이후 고객수가 크게 늘어 기존 11자리 체계에서 12자리 체계로의 변경이 필요했습니다만, 신계좌로 전환하여도 기존 구계좌번호를 이용하시는데 (기존 자동이체 및
급여입금 등) 전혀 문제가 없슴을 말씀드립니다.
-----------------------------------------------
2. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
context: 인터넷뱅킹에 로그인하신 후 "공과금/법원 > 공과금센터" 페이지에 가시면 "지로자동이체 등록" 메뉴가 있습니다. 해당 메뉴를 통하여 "전기요금, 전화요금,
국민연금, 국민건강보험료 등을 포함하여 각종 지로요금"을 모두 자동이체 등록하실 수 있습니다.
-----------------------------------------------
3. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
context: 인터넷뱅킹 가입과 홈페이지 회원은 개별로 운영되고 있습니다.
인터넷뱅킹만 가입을 하셨다면 별도로 홈페이지에 회원가입 후 홈페이지 로그인이 가능합니다.
※ 영업점에서 인터넷뱅킹 가입 시 전계좌조회서비스 가입하신 고객은 홈페이지 상에서 이용자비밀번호 등록 후 이용가능합니다.
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
-----------------------------------------------
4. context to be fed into FM(e.g. Claude-v2)

### 챗봇 (ConversationalRetrievalChain) 의로 질문

In [36]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

새로운 질문으로만 대답하세요.


Human: 이전 대화를 고려하여 질문을 어떻게 하시겠습니까?: {question}


Assistant: Question:""")


# recreate the Claude LLM with more tokens to sample - this provide longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_fsi.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}


Human: <q></q> XML 태그 내의 질문에 답하려면 최대 3개의 문장을 사용하세요.

<q>{question}</q>

답변에 XML 태그를 사용하지 마세요. 답변이 context에 없으면 "죄송합니다. 문맥에서 답을 찾을 수 없어서 모르겠습니다."라고 말합니다.

Assistant:""")

Let's start another chat. Feel free to ask the following questions:

신한은행 고객센터 전화 번호 알려줘?


In [37]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

# 7.Next Action

프롬프트를 "조정"하여 다소 자세한 답변을 얻을 수 있습니다. 예를 들어 문장 수를 변경하거나 해당 명령을 모두 제거해 보세요. 전체 답변을 얻으려면 'max_tokens_to_sample' 수(예: 1000 또는 2000)를 변경해야 할 수도 있습니다.

### 요약
- 이 데모에서는 Claude LLM을 사용하여 다음 패턴으로 대화형 인터페이스를 만들었습니다.

1. 챗봇(기본 - 맥락 없음)

2. 프롬프트 템플릿(Langchain)을 이용한 챗봇

3. 페르소나를 갖춘 챗봇
4. 맥락을 갖춘 챗봇